In [ ]:
!curl -O https://www.antlr.org/download/antlr-4.13.2-complete.jar &> /dev/null

In [ ]:
!pip install -U antlr4-python3-runtime &> /dev/null

# Creación del Grammar

In [ ]:
%%writefile GameScript.g4
grammar GameScript;

script: command+ EOF;

command:
      moveCommand
    | jumpCommand
    | attackCommand
    | waitCommand
    ;

moveCommand: 'move' direction amount NEWLINE;
jumpCommand: 'jump' NEWLINE;
attackCommand: 'attack' target NEWLINE;
waitCommand: 'wait' amount 'seconds' NEWLINE;


direction: 'left' | 'right' | 'up' | 'down';
target: 'enemy' | 'boss' | 'object';
amount: INT;

INT: [0-9]+;
NEWLINE: ('\r'? '\n' | ';')+;
WS: [ \t]+ -> skip;



Writing GameScript.g4


In [ ]:
!java -cp .:antlr-4.13.2-complete.jar org.antlr.v4.Tool GameScript.g4 -no-listener -visitor -Dlanguage=Python3

## Documento para probar el código

In [6]:
%%writefile test.tp
move right 5
jump
attack enemy

wait 2 seconds

move left 3;
attack boss


Writing test.tp


## Archivo visitor

In [7]:
import sys
from antlr4 import *
from GameScriptLexer import GameScriptLexer
from GameScriptParser import GameScriptParser
from GameScriptVisitor import GameScriptVisitor

class GameActionVisitor(GameScriptVisitor):

    # Método para visitar un nodo de tipo 'moveCommand'
    def visitMoveCommand(self, ctx:GameScriptParser.MoveCommandContext):
        # Obtenemos el texto de los nodos hijos 'direction' y 'amount'
        direction = ctx.direction().getText()
        amount = ctx.amount().getText()
        print(f"Action: MOVE, Direction: {direction}, Steps: {amount}")
        return self.visitChildren(ctx)

    # Método para visitar un nodo de tipo 'jumpCommand'
    def visitJumpCommand(self, ctx:GameScriptParser.JumpCommandContext):
        print("Action: JUMP")
        return self.visitChildren(ctx)

    # Método para visitar un nodo de tipo 'attackCommand'
    def visitAttackCommand(self, ctx:GameScriptParser.AttackCommandContext):
        target = ctx.target().getText()
        print(f"Action: ATTACK, Target: {target}")
        return self.visitChildren(ctx)

    # Método para visitar un nodo de tipo 'waitCommand'
    def visitWaitCommand(self, ctx:GameScriptParser.WaitCommandContext):
        amount = ctx.amount().getText()
        print(f"Action: WAIT, Duration: {amount} seconds")
        return self.visitChildren(ctx)


def main(argv):
    """
    Función principal que lee el archivo de script, lo analiza y lo procesa.
    """
    if len(argv) < 2:
        print("Uso: python driver.py <ruta_del_archivo_script>")
        return

    # Creamos un stream de caracteres a partir del archivo de entrada
    input_stream = FileStream(argv[1])

    # Creamos el lexer, que convierte los caracteres en tokens
    lexer = GameScriptLexer(input_stream)

    # Creamos un stream de tokens a partir del lexer
    stream = CommonTokenStream(lexer)

    # Creamos el parser, que consume los tokens y construye el árbol de análisis
    parser = GameScriptParser(stream)

    # Indicamos al parser que comience a analizar desde la regla 'script'
    tree = parser.script()

    # Verificamos si hubo errores de sintaxis durante el análisis
    if parser.getNumberOfSyntaxErrors() > 0:
        print("El script contiene errores de sintaxis.")
    else:
        print("Script analizado exitosamente! Ejecutando acciones...")
        print("-" * 20)
        # Creamos una instancia de nuestro visitor y lo usamos para recorrer el árbol
        visitor = GameActionVisitor()
        visitor.visit(tree)
        print("-" * 20)
        print("Ejecución finalizada.")

if __name__ == '__main__':
    main(["driver.py", "test.tp"])

Script analizado exitosamente! Ejecutando acciones...
--------------------
Action: MOVE, Direction: right, Steps: 5
Action: JUMP
Action: ATTACK, Target: enemy
Action: WAIT, Duration: 2 seconds
Action: MOVE, Direction: left, Steps: 3
Action: ATTACK, Target: boss
--------------------
Ejecución finalizada.
